In [ ]:
from algorithms import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from itertools import product
from seaborn import heatmap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [ ]:
higgs_data = pd.read_csv("./data/HIGGS-002.zip", header=None)
higgs_data.head()

In [ ]:
higgs_data.describe()

In [ ]:
higgs_data[0].value_counts(dropna=False)

In [ ]:
heatmap(higgs_data.corr())
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
higgs_data["intercept"] = 1
feature_names = list(higgs_data.drop(columns=0))
features = higgs_data.drop(columns=0).values
features = MinMaxScaler().fit_transform(features)
target = higgs_data[0].copy()

(
    X, 
    X_test, 
    y, 
    y_test
)=train_test_split(features, target.values, test_size=0.25, random_state=3136, stratify=target.values)


# # Oversample to balance the data
# n0 = y.shape[0] - y.sum()
# idx1 = np.array(range(y.shape[0]))[(y==1).flatten()]
# new_idx = np.random.choice(idx1, size=n0, replace=True)

# X = np.concatenate([X[(y == 0).flatten()], X[new_idx]])
# y = np.concatenate([y[(y == 0).flatten()], y[new_idx]])

# Logistic Regression

## Determine Optimization Parameters for Momentum SGD

In [ ]:
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

opt_candidates={
    "lr":  [0.1, 1, 5, 10],
    "batch_size": [16, 64, 256, 1024],
    "momentum": [0, 0.5, 0.9],
    "max_epoch": [25],
}

mu = 0

results = {}
for params in product(*opt_candidates.values()):
    lr, batch_size, momentum, max_epoch = params
    param_key = "_".join(str(x) for x in params)
    
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w = MomentumStochasticGradient(
            X=X_train,
            y=y_train,
            mu=mu,
            lr=lr,
            moment=momentum,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        preds = sigmoid(X_val@w)
        loss = neg_log_lik(preds, y_val, w, mu)
        cv_results.append(loss)
        
    results[param_key] = np.mean(cv_results)
    

best_params = min(results, key=lambda key: results[key])
print(f"BEST: {best_params}")
results

### Determine Regularization Parameter

In [ ]:
mu_candidates = np.arange(0, 0.01, 0.001)
lr, batch_size, momentum, max_epoch = [float(x) if "." in x else int(x) for x in best_params.split("_")]
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

results = {}
for mu in mu_candidates:
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        w = MomentumStochasticGradient(
            X=X_train,
            y=y_train,
            mu=mu,
            lr=lr,
            moment=momentum,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )

        probs = sigmoid(X_val@w) #> 0.5
        loss = roc_auc_score(y_val, probs)
        cv_results.append(loss)

    results[str(mu)] = np.mean(cv_results)

selected_mu = float(max(results, key=lambda key: results[key]))
print("BEST MU: ", selected_mu)
results

In [ ]:
w

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty="none", max_iter=1000)
logreg.fit(X, y)
test_pred = logreg.predict(X_test)
print("Balanced Accuracy", balanced_accuracy_score(y_test, test_pred))

### Test Results

In [ ]:
w = MomentumStochasticGradient(
    X=X,
    y=y,
    mu=selected_mu,
    lr=lr,
    moment=momentum,
    batch_size=batch_size,
    max_epoch=max_epoch,
)
test_prob = sigmoid(X_test@w)
#test_pred = (test_prob > (y_train.sum()/y_train.shape[0])).astype(int)
test_pred = (test_prob > 0.5).astype(int)
print("\nTEST LOSS: ",neg_log_lik(test_prob, y_test, w, mu=selected_mu))
print("Balanced Accuracy", balanced_accuracy_score(y_test, test_pred))

In [ ]:
for coef, feature in  sorted(zip(w, feature_names)):
    print(feature, coef)

In [ ]:
fig = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, test_pred))
fig.plot(cmap="Reds")
plt.show()

## Determine Optimization Parameters for AdaGrad

In [ ]:
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

opt_candidates={
    "lr":  [0.1, 1, 5, 10],
    "batch_size": [16, 64, 256, 512, 1024],
    "max_epoch": [40],
}

mu = 0

results = {}
for params in product(*opt_candidates.values()):
    lr, batch_size, max_epoch = params
    param_key = "_".join(str(x) for x in params)
    
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w = AdaGrad(
            X=X_train,
            y=y_train,
            mu=mu,
            lr=lr,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        preds = sigmoid(X_val@w)
        loss = neg_log_lik(preds, y_val, w, mu)
        cv_results.append(loss)
        
    results[param_key] = np.mean(cv_results)
    

best_params = min(results, key=lambda key: results[key])
print(f"BEST: {best_params}")
results

### Determine Regularization Parameter

In [ ]:
mu_candidates = np.arange(0, 0.01, 0.001)
lr, batch_size, max_epoch = [int(x) for x in best_params.split("_")]
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

results = {}
for mu in mu_candidates:
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        w = AdaGrad(
            X=X_train,
            y=y_train,
            mu=mu,
            lr=lr,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )

        probs = sigmoid(X_val@w) #> 0.5
        loss = roc_auc_score(y_val, probs)
        cv_results.append(loss)

    results[str(mu)] = np.mean(cv_results)

selected_mu = float(max(results, key=lambda key: results[key]))
print("BEST MU: ", selected_mu)
results

### Test Results

In [ ]:
w = AdaGrad(
    X=X,
    y=y,
    mu=selected_mu,
    lr=lr,
    batch_size=batch_size,
    max_epoch=max_epoch,
)
test_prob = sigmoid(X_test@w)
#test_pred = (test_prob > (y_train.sum()/y_train.shape[0])).astype(int)
test_pred = (test_prob > 0.5).astype(int)
print("\nTEST LOSS: ",neg_log_lik(test_prob, y_test, w, mu=selected_mu))
print("Balanced Accuracy", balanced_accuracy_score(y_test, test_pred))

In [ ]:
for coef, feature in  sorted(zip(w, feature_names)):
    print(feature, coef)

In [ ]:
fig = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, test_pred))
fig.plot(cmap="Reds")
plt.show()

# SVM

## Determine Optimization Parameters for Momentum Stochastic Subgradient

In [ ]:
# Drop intercept
X = X[:, :-1]
y_svm = y.copy()
y_svm[y_svm == 0 ] = -1
y = y_svm.reshape(-1,1)
# Set large C for strict model
C = 1e18

splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

opt_candidates={
    "lr":  [0.1, 1, 10],
    "batch_size": [16, 64, 256, 1024],
    "momentum": [0, 0.5, 0.9],
    "max_epoch": [25],
}

results = {}
for params in product(*opt_candidates.values()):
    lr, batch_size, momentum, max_epoch = params
    param_key = "_".join(str(x) for x in params)
    
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w0, w = MomentumSubGradient(
            X=X_train,
            y=y_train,
            C=C,
            lr=lr,
            moment=momentum,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        loss = HingeLossl2(C, w, w0, X_val, y_val)
        cv_results.append(loss)
        
    results[param_key] = np.mean(cv_results)
    
best_params = min(results, key=lambda key: results[key])
print(f"BEST: {best_params}")
results

## Tune Parameter C

In [ ]:
splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=3136)

C_candidates = [1e-1, 1e0,1e1, 1e2, 1e3, 1e4]
lr, batch_size, momentum, max_epoch = [float(x) if "." in x else int(x) for x in best_params.split("_")]


results = {}
for C in C_candidates:
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w0, w = MomentumSubGradient(
            X=X_train,
            y=y_train,
            C=C,
            lr=lr,
            moment=momentum,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        pred = (w0+X_val@w) > 0
        y_val[y_val == -1] = 0
        loss = balanced_accuracy_score(y_val, pred)
        cv_results.append(loss)
        
    results[str(C)] = np.mean(cv_results)
    
selected_C = float(max(results, key=lambda key: results[key]))
print("BEST C: ", selected_C)
results

In [ ]:
selected_C = 10000
lr, batch_size, momentum, max_epoch = 0.1, 16, 0.5, 25
w0, w = MomentumSubGradient(
            X=X,
            y=y,
            C=selected_C,
            lr=lr,
            moment=momentum,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
test_pred = (w0+X_test[:,:-1]@w) > 0
#print("\nTEST LOSS: ",HingeLossl2(selected_C, w, w0, X_test[:,:-1], y_test))
print("Balanced Accuracy", balanced_accuracy_score(y_test, test_pred))

In [ ]:
for coef, feature in  sorted(zip(w, feature_names)):
    print(feature, coef)

In [ ]:
fig = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, test_pred))
fig.plot(cmap="Reds")
plt.show()

## Determine Optimization Parameters for AdaSubgradient

In [ ]:
# Set large C for strict model
C = 1e18

splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

opt_candidates={
    "lr":  [0.1, 1, 5, 10],
    "batch_size": [16, 64, 256, 512, 1024],
    "max_epoch": [40],
}

results = {}
for params in product(*opt_candidates.values()):
    lr, batch_size, max_epoch = params
    param_key = "_".join(str(x) for x in params)
    
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w0, w = AdaSubGradient(
            X=X_train,
            y=y_train,
            C=C,
            lr=lr,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        loss = HingeLossl2(C, w, w0, X_val, y_val)
        cv_results.append(loss)
        
    results[param_key] = np.mean(cv_results)
    
best_params = min(results, key=lambda key: results[key])
print(f"BEST: {best_params}")
results

### Tune Parameter C

In [ ]:
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3136)

C_candidates = list(np.arange(100, 1000, 25))  + [1e1, 1, 1e-1, 1e-2]
lr, batch_size, max_epoch = [float(x) if "." in x else int(x) for x in best_params.split("_")]


results = {}
for C in C_candidates:
    cv_results = []
    for train_index, val_index in splitter.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        w0, w = AdaSubGradient(
            X=X_train,
            y=y_train,
            C=C,
            lr=lr,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
        
        pred = (w0+X_val@w) > 0
        y_val[y_val == -1] = 0
        loss = balanced_accuracy_score(y_val, pred)
        cv_results.append(loss)
        
    results[str(C)] = np.mean(cv_results)
    
selected_C = float(max(results, key=lambda key: results[key]))
print("BEST C: ", selected_C)
results

### Test Results

In [ ]:
w0, w = AdaSubGradient(
            X=X,
            y=y,
            C=selected_C,
            lr=lr,
            batch_size=batch_size,
            max_epoch=max_epoch,
        )
test_pred = (w0+X_test[:,:-1]@w) > 0
print("\nTEST LOSS: ",HingeLossl2(selected_C, w, w0, X_test[:,:-1], y_test))
print("Balanced Accuracy", balanced_accuracy_score(y_test, test_pred))

In [ ]:
for coef, feature in  sorted(zip(w, feature_names)):
    print(feature, coef)

In [ ]:
fig = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, test_pred))
fig.plot(cmap="Reds")
plt.show()